# NBA Player Data Analysis Project

## Project Outline

1. **Introduction**
   - Overview of the project
   - Goals and objectives

2. **Data Collection**
   - Import necessary libraries
   - Fetch player data from the NBA API

3. **Data Processing**
   - Load data into a Pandas DataFrame
   - Data cleaning and transformation

4. **Data Analysis**
   - Exploratory data analysis (EDA)
   - Visualizations

5. **Conclusion**
   - Summary of findings
   - Future work


## 0. Environment

### Python

This project is written in Python, which means that Python must be installed in your environment to run the project. The minimum supported version is 3.10.

#### Windows

You can use the Windows package manager `winget`, or the [installer](https://www.python.org/downloads/windows/) from the website.
```powershell
# you can change the version in the package name to your desired version
winget install Python.Python.3.12
```

#### MacOS
Python is already installed by default on recent versions of MacOS. If you have an older version that is not supported, you can use the [Homebrew](https://brew.sh/) package manager to install it, or the [installer](https://www.python.org/downloads/macos/) from the website.
```zsh
brew install python
```

#### Linux
Python is already installed by default on most distributions of Linux. If it isn't, you can use your distribution's package manager to install Python.

### Virtual Environment

It's generally recommended that you use a virtual environment (or venv) for this project. That way, all dependencies can be installed for the project without affecting the rest of your system. You can create a venv with Python:

```bash
python -m venv .venv
```

To activate the virtual environment in your shell, you can use the following commands.

On Windows:

```powershell
.venv\Scripts\activate
```

On other operating systems:

```bash
.venv/bin/activate
```

### Dependencies

This project uses [Poetry](https://python-poetry.org/) to manage its dependencies. You can install the dependencies with the `poetry` command:

`poetry install`

If you don't want to use Poetry, a `requirements.txt` is also provided. You can install this using `pip`:

`pip install -r requirements.txt`

### Imports

In [ ]:
import requests
import json
import time
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
from typing import Dict

### Environment Variables

We will load all our environment variables from a `.env` file, if one is provided.

If database information is provided, all dataframes used for analysis are uploaded to it. We use [Microsoft SQL Server](https://www.microsoft.com/en-us/sql-server/sql-server-downloads) by default but any kind of database is supported.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
DB_TYPE = os.getenv("DB_TYPE", "sqlserver")
DB_USER = os.getenv("DB_USER", "sqladmin")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "1433")
DB_NAME = os.getenv("DB_NAME", "dataframes")
DB_DRIVER = os.getenv("DB_DRIVER", "ODBC Driver 18 for SQL Server")

### Presentation

By default, Pandas dataframes are truncated when they are printed. We want to be able to view all of the data at once, so we embed the dataframe in a scrollable element.

In [ ]:
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

def custom_scrollable_display(df: pd.DataFrame, max_height=400):
    """
    Custom display function to render DataFrames as scrollable elements.
    
    Parameters:
    - df: The DataFrame to display.
    - max_height: The maximum height of the scrollable area in pixels.
    """
    style = f"""
    <style>
    .scrollable-dataframe {{
        display: inline-block;
        white-space: nowrap;
        overflow-x: scroll;
        max-height: {max_height}px;
        overflow-y: scroll;
    }}
    </style>
    """
    display(HTML(style + f'<div class="scrollable-dataframe">{df.to_html()}</div>'))

def custom_display_hook(df):
    custom_scrollable_display(df)
    return ""

# hook up the custom display function to the automatic printer
InteractiveShell.instance().display_formatter.formatters['text/html'].for_type(pd.DataFrame, custom_display_hook);


### Pre-Commit Hooks (Developer Only)

This notebook uses `nbstripout` to strip notebook output from Git commits. If you are committing code, please run the following command to set up the Git filter.

Poetry is required for the pre-commit hooks, so make sure it is installed before you commit code. You will also need to add the plugin `poetry-plugin-export` in order to run the export hook.
```bash
poetry self add poetry-plugin-export
```

In [ ]:
!nbstripout --install
!pre-commit install

## 1. Data Collection

### Fetch Player Data from NBA API

`nba_api` provides static player and team information, which we will download here so that we can reuse it without requesting the API unnecessarily.

In [ ]:
from nba_api.stats.static import players, teams
PLAYERS_LIST_FILE = "../data/players_list.json"
TEAMS_LIST_FILE = "../data/teams_list.json"

if not os.path.exists(PLAYERS_LIST_FILE):
    players_list = players.get_players()
    with open(PLAYERS_LIST_FILE, "w") as f:
        json.dump(players_list, f)
else:
    with open(PLAYERS_LIST_FILE, "r") as f:
        players_list = json.load(f)
if not os.path.exists(TEAMS_LIST_FILE):
    teams_list = teams.get_teams()
    teams.get_wnba_teams
    with open(TEAMS_LIST_FILE, "w") as f:
        json.dump(teams_list, f)
else:
    with open(TEAMS_LIST_FILE, "r") as f:
        teams_list = json.load(f)

### Fetch Game Data



We're only interested in games that are either in the regular season or in the playoffs. We'll add an enum to distinguish the type of game and use it to differentiate them.

In [ ]:
from enum import Enum
class SeasonType(Enum):
    PRESEASON = 1
    REGULAR_SEASON = 2
    ALL_STAR = 3
    PLAYOFFS = 4
    PLAY_IN = 5
    NBA_CUP = 6
class Season():
    def __init__(self, season_id: int) -> None:
        season_id_str = str(season_id)
        self.season_type = SeasonType(int(season_id_str[0]))
        self.season_year = int(season_id_str[1:])

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import LeagueIDNullable, LeagueID
START_SEASON = 2012
END_SEASON = 2023
GAMES_LIST_FILE = "../data/games_list.csv"
if os.path.exists(GAMES_LIST_FILE):
    games_list: pd.DataFrame = pd.read_csv(GAMES_LIST_FILE)
else:
    games_list = pd.DataFrame()
    for season in range(START_SEASON, END_SEASON + 1):
        # put season into the correct form e.g. 2023 -> 2023-24
        season_str = f"{season}-{str(season + 1)[2:]}"
        print(f"Fetching games for season: {season_str}", end="\r")
        gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season_str, league_id_nullable=LeagueIDNullable.nba)
        games = gamefinder.get_data_frames()[0]
        games_list = pd.concat([games_list, games], ignore_index=True)
        time.sleep(0.6)
    games_list.to_csv(GAMES_LIST_FILE, index=False)
# games_list["SEASON_ID"].unique()
games_list.head()

### Fetch Play by Plays

In [ ]:
from nba_api.stats.endpoints import playbyplayv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
PBP_LIST_FILE = "../data/pbp_list.csv"
if os.path.exists(PBP_LIST_FILE):
    pbp_list = pd.read_csv(PBP_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    pbp_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching play by play for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                pbpfinder = playbyplayv3.PlayByPlayV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a play by play", file=f)
                err = True
                break
        if err:
            continue
        pbp = pbpfinder.get_data_frames()[0]
        pbp_list = pd.concat([pbp_list, pbp], ignore_index=True)
        time.sleep(0.6)
    pbp_list.to_csv(PBP_LIST_FILE, index=False)
pbp_list.head()


## 2. Data Processing

### Clock

We will define a class to store the clock time.

In [ ]:
class Clock():
    def __init__(self, clock) -> None:
        minutes = int(clock[2:4])
        seconds = int(clock[5:7])
        self.total_seconds = minutes * 60 + seconds

    def __repr__(self) -> str:
        minutes, seconds = divmod(self.total_seconds, 60)
        return f"{minutes:02}:{int(seconds):02}"

    def __hash__(self) -> int:
        return hash(self.total_seconds)

    def __int__(self):
        return self.total_seconds

    def __eq__(self, other):
        if isinstance(other, Clock):
            return self.total_seconds == other.total_seconds
        return self.total_seconds == other

    def __lt__(self, other):
        if isinstance(other, Clock):
            return self.total_seconds < other.total_seconds
        return self.total_seconds < other

    def __le__(self, other):
        return self < other or self == other

    def __gt__(self, other):
        return not self <= other

    def __ge__(self, other):
        return not self < other

    def __add__(self, other):
        if isinstance(other, Clock):
            return Clock(self.total_seconds + other.total_seconds)
        return Clock(self.total_seconds + other)

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        if isinstance(other, Clock):
            return Clock(self.total_seconds - other.total_seconds)
        return Clock(self.total_seconds - other)

    def __rsub__(self, other):
        return Clock(other - self.total_seconds)


and transform the clock time to it.

In [ ]:
def to_clock(clock):
    if isinstance(clock, Clock):
        return clock
    else:
        return Clock(clock)
pbp_list["clock"] = pbp_list["clock"].apply(to_clock)
pbp_list["clock"].head()

## 3. Data Analysis

Our analysis will be focused on when shots with assists are being made, so we will isolate this dataframe for the relevant data.

In [ ]:
made_shots = pbp_list.loc[pbp_list["actionType"] == "Made Shot"].reset_index(drop=True)

df = pd.DataFrame()

df["clock"] = made_shots["clock"]

# we're not using the number numerically, so we can mark this as a categorical variable to save on memory and storage
df["period"] = pd.Categorical(made_shots["period"])

df["has_assist"] = made_shots["description"].str.contains("AST")

df.head()

## 4. Data Reporting

To create our final report from our analysis, we will be using Power BI. We have a Microsoft SQL Server database that our Power BI report will import the tables from. If no database is available, the dataframes will instead export as an Excel spreadsheet, which can be manually uploaded to Power BI.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection
DATABASE_URL = f"{DB_TYPE}://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}{'?driver=' if DB_DRIVER else ''}{DB_DRIVER.replace(' ', '+') if DB_DRIVER else ''}"

engine = create_engine(DATABASE_URL)

with engine.connect() as connection:
    display("Connection to the database was successful!")

def upload_dataframes(dfs: Dict[str, pd.DataFrame]) -> None:
    inspector = reflection.Inspector.from_engine(engine)
    existing_tables = inspector.get_table_names()

    for table_name, df in dfs.items():
        # check if table already exists
        if table_name in existing_tables:
            # if there are no changes to the table, do not write to it
            existing_df = pd.read_sql_table(table_name, engine)
            if df.shape == existing_df.shape and df.equals(existing_df):
                print(f"No changes detected for table {table_name}. Skipping upload.")
                continue
        else:
            print(f"Table {table_name} does not exist. Creating a new one.")
        df.to_sql(table_name, engine, if_exists="replace", index=False)
        print(f"Uploaded dataframe to table {table_name}.")

upload_dataframes({"assists_in_the_clutch": df})